# Resolve Schema

**input**
```
+schema_path: str
```

**methods**
```
+read_json(schema_path: str): dict
+split_json(schema: dict): list
+resolver(entity1: dict, entity2: dict): dict
+resolve_defs(terms: dict, defs: dict) : dict
+ node_order(schema: dict): list
+resolve_nodes(nodeList: list, splitJsonList: list): list
+recombine_nodes(resolvedList: list) : dict
```

In [ ]:
import gen3validator
from gen3validator.logging_config import setup_logging
setup_logging()

In [ ]:
# pulling manifest
!aws s3 cp s3://ausdiab-data-receive-bucket/data/2025-01-31_AusDiab_ACDC_Data_Transfer/manifest.xlsx ../data/restricted/ausdiab_lipid_manifest.xlsx

In [ ]:
# pulling schema
!aws s3 cp s3://gen3schema-cad-staging-biocommons.org.au/cad.json ../schema/gen3_schema.json

## Reading in xlsx data and writing to json
- xlsx data comes from xlsx manifest file created from acdc_submission_template

In [6]:
# ResolverClass = gen3validator.ResolveSchema(schema_path = "../schema/gen3_test_schema.json")
xlsxData = gen3validator.ParseXlsxMetadata(xlsx_path = "/Users/harrijh/projects/gen3-data-validator/data/lipid_metadata_example.xlsx", skip_rows=1)
xlsxData.parse_metadata_template()
xlsxData.write_dict_to_json(xlsx_data_dict=xlsxData.xlsx_data_dict, output_dir="/Users/harrijh/projects/gen3-data-validator/data/restricted/lipid_metadata_example")

## Testing Linkage

In [8]:
Data = gen3validator.ParseData(data_folder_path = "/Users/harrijh/projects/gen3-data-validator/data/restricted/lipid_metadata_example")

In [ ]:
Data.file_path_list

In [ ]:
Data = gen3validator.ParseData(data_folder_path = "/Users/harrijh/projects/gen3-data-validator/data/restricted/lipid_metadata_example")
Resolver = gen3validator.ResolveSchema(schema_path = "../schema/gen3_schema.json")
Resolver.resolve_schema()
Linkage = gen3validator.Linkage(schema_resolver = Resolver, data_parser = Data)

In [3]:
Resolver = gen3validator.ResolveSchema(schema_path = "../schema/gen3_schema.json")

In [ ]:
Resolver.nodes

You can also bypass the data attribute in the linkage class and input your own data_map and config_map

In [7]:
# # Using the Linkage class which has the resolved schema with custom data and config
# config_map = {
#     "samples": {"primary_key": "sample_id", "foreign_key": "subject_id"},
#     "files": {"primary_key": "file_id", "foreign_key": "sample_id"},
#     "subjects": {"primary_key": "subject_id", "foreign_key": "project_id"},
#     "project": {"primary_key": "project_id", "foreign_key": None}
# }

# data_map = {
#     "samples": [
#         {"sample_id": "sample_1", "subject_id": "subject_9"},
#         {"sample_id": "sample_2", "subject_id": "subject_3"},  # Invalid FK
#         {"sample_id": "sample_3", "subject_id": "subject_4"}, # Invalid FK
#         {"sample_id": "sample_4", "subject_id": "subject_5"} # Invalid FK
#     ],
#     "files": [
#         {"file_id": "file_1", "sample_id": "sample_1"},
#         {"file_id": "file_2", "sample_id": "sample_27"}  # Invalid FK
#     ],
#     "subjects": [
#         {"subject_id": "subject_1", "project_id": "project_1"},  
#         {"subject_id": "subject_2", "project_id": "project_2"}, # Missing project 2
#     ],
#     "project": [
#         {"project_id": "project_1"}
#     ]
# }

# Linkage.validate_links(data_map, config_map)

# validation prototype


## Creating the validation class

In [2]:
import gen3validator

resolver = gen3validator.ResolveSchema(schema_path = "../schema/gen3_schema.json")
resolver.resolve_schema()
data = gen3validator.ParseData(data_folder_path = "../data/restricted/lipid_metadata_example")
validator = gen3validator.Validate(data_map=data.data_dict, resolved_schema=resolver.schema_resolved)

In [ ]:
validator.validate_schema()

In [ ]:
validator.make_keymap()

In [ ]:
data.data_dict

### Getting nested validation results
- returns a nested dictionary by entity/data node then by the row/index number, and then the validation objects

In [ ]:
validation_dict = validator.validation_result
validation_dict

In [ ]:
validator.list_entities()

In [ ]:
validator.list_index_by_entity("lipidomics_assay")

You can pull out a validation results for a specific entity with

In [ ]:
validator.pull_entity("lipidomics_assay")

You can pull validation results for a specific entity and then a specific index / row

In [ ]:
validator.pull_index_of_entity("lipidomics_assay", "index_1")

# Getting validation stats

In [ ]:
validate_stats = gen3validator.ValidateStats(validator)
stats_df = validate_stats.summary_stats()
stats_df

# Creating validation summary data

In [ ]:
Summary = gen3validator.ValidateSummary(validator) 
flattened_results_dict = Summary.flatten_validation_results()
flattened_results_dict

### Converting flattened dict to pandas

In [ ]:
flatten_summary_pd = Summary.flattened_results_to_pd()
flatten_summary_pd

### Collapsing flattened dict to pandas
- This collapsed data frame summarises common validation errors

In [ ]:
collapse_df = Summary.collapse_flatten_results_to_pd()
collapse_df

# Writing validation results to folder

In [ ]:
import os
output_dir = "../data/restricted/ausdiab_lipid_metadata/validation/"
os.makedirs(output_dir, exist_ok=True)


def write_dict_to_json(input_dict, output_dir, filename:str):
    with open(f"{output_dir}/{filename}.json", "w") as f:
        json.dump(input_dict, f)
    print(f"JSON files written to {output_dir}")

write_dict_to_json(validation_dict, output_dir, "validation_dict")
write_dict_to_json(flattened_results_dict, output_dir, "flattened_results_dict")

# Writing pandas df
stats_df.to_csv(f"{output_dir}/stats_df.csv")
flatten_summary_pd.to_csv(f"{output_dir}/flatten_summary_pd.csv")
collapse_df.to_csv(f"{output_dir}/collapse_df.csv")


In [ ]:
# Use this for writing tests

sample_validation_results = {
    'sample': [
        [
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'10' is not of type 'integer'"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_provider',
                'schema_path': 'properties.sample_provider.enum',
                'validator': 'enum',
                'validator_value': ['Baker', 'USYD', 'UMELB', 'UQ'],
                'validation_error': "45 is not one of ['Baker', 'USYD', 'UMELB', 'UQ']"
            },
            {
                'index': 0,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'Autoclave' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'freeze_thaw_cycles',
                'schema_path': 'properties.freeze_thaw_cycles.type',
                'validator': 'type',
                'validator_value': 'integer',
                'validation_error': "'76' is not of type 'integer'"
            },
            {
                'index': 1,
                'validation_result': 'FAIL',
                'invalid_key': 'sample_storage_method',
                'schema_path': 'properties.sample_storage_method.enum',
                'validator': 'enum',
                'validator_value': [
                    'not stored',
                    'ambient temperature',
                    'cut slide',
                    'fresh',
                    'frozen, -70C freezer',
                    'frozen, -150C freezer',
                    'frozen, liquid nitrogen',
                    'frozen, vapor phase',
                    'paraffin block',
                    'RNAlater, frozen',
                    'TRIzol, frozen'
                ],
                'validation_error': "'In the Pantry' is not one of ['not stored', 'ambient temperature', 'cut slide', 'fresh', 'frozen, -70C freezer', 'frozen, -150C freezer', 'frozen, liquid nitrogen', 'frozen, vapor phase', 'paraffin block', 'RNAlater, frozen', 'TRIzol, frozen']"
            }
        ],
        [
            {
                'index': 2,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ],
        [
            {
                'index': 3,
                'validation_result': 'PASS',
                'invalid_key': None,
                'schema_path': None,
                'validator': None,
                'validator_value': None,
                'validation_error': None
            }
        ]
    ]
}


